In [2]:
#! pip3 install transformers
import transformers
print(transformers.__version__)

4.9.2


In [176]:
# This flag is the difference between SQUAD v1 or 2 (if you're using another dataset, it indicates if impossible
# answers are allowed or not).
squad_v2 = False
model_checkpoint='./xlm-roberta'
batch_size = 4

In [2]:
#!pip3 install datasets
from datasets import load_dataset
import datasets
print (datasets.__version__)

1.11.0


In [173]:
#datasets = load_dataset('csv', data_files='./data/train.csv')

datasets = load_dataset('csv', data_files={'train': './data/train.csv',
                                          'validation': './data/train.csv'})
                                          #'test':'./data/test.csv'})

Using custom data configuration default-f17c46892de75ae9
Reusing dataset csv (/home/rao/.cache/huggingface/datasets/csv/default-f17c46892de75ae9/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)


In [177]:
datasets['validation']

Dataset({
    features: ['id', 'context', 'question', 'answer_text', 'answer_start', 'language'],
    num_rows: 1114
})

In [178]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [179]:
#show_random_elements(dataset["train"])

In [180]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

loading configuration file ./xlm-roberta/config.json
Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

Didn't find file ./xlm-roberta/added_tokens.json. We won't load it.
Didn't find file ./xlm-roberta/special_tokens_map.json. We won't load it.
loading file ./xlm-roberta/sentencepiece.bpe.model
loading file ./xlm-roberta/tokenizer.json
loading file None
loading file None
l

In [181]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [182]:
tokenizer("एशिया का कुल क्षेत्रफल कितना है?","एशिया का कुल क्षेत्रफल कितना है?")

{'input_ids': [0, 151677, 641, 40696, 10386, 66646, 153261, 460, 32, 2, 2, 151677, 641, 40696, 10386, 66646, 153261, 460, 32, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [183]:
max_length = 256 # The maximum length of a feature (question and context)
doc_stride = 32 # The authorized overlap between two part of the context when splitting it is needed.

In [184]:

for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

In [185]:
len(tokenizer(example["question"], example["context"])["input_ids"])

1504

In [186]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

256

In [187]:
 example["question"]

'மனித உடலில் எத்தனை எலும்புகள் உள்ளன?'

In [188]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation='only_second',
    return_overflowing_tokens=True,
    stride=doc_stride
)

In [189]:
print (
[len(x) for x in tokenized_example["input_ids"]])
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))


[256, 256, 256, 256, 256, 256, 244]
<s> மனித உடலில் எத்தனை எலும்புகள் உள்ளன?</s></s> ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின்வரும் 206 (மார்பெலும்பு மூன்று பகுதிகளாகக் கருதப்பட்டால் 208) எண்ணிக்கையான எலும்புகளைக் கொண்டிருக்கும். இந்த எண்ணிக்கை உடற்கூட்டியல் வேறுபாடுகளைப் பொறுத்து மாறுபடக்கூடும். எடுத்துக்காட்டாக, மிகக் குறைந்த எண்ணிக்கையான மனிதர்களில், ஒரு மேலதிக விலா எலும்பு (கழுத்துவில்) அல்லது ஒரு மேலதிகமான கீழ் முதுகெலும்பு காணப்படுவதுண்டு; இணைந்த சில எலும்புகளைத் தனி எலும்பாகக் கருதாவிடின், ஐந்து இணைந்த திருவெலும்பு; மூன்று (3 - 5) குயிலலகு எலும்புகள் சேர்ந்து 26 எண்ணிக்கையிலான முதுகெலும்புகள் 33 ஆகக் கருதப்படலாம். மனித மண்டையோட்டில் 22 எலும்புகள் (காதுச் சிற்றென்புகளைத் தவிர) உள்ளன; இவை எட்டு மண்டையறை (cranium) எலும்புகளாகவும் 14 முக எலும்புகளாகவும் (facial bones) பிரிக்கப்பட்டுள்ளன. (தடித்த எண்கள் அருகிலுள்ள படத்தில் காணும் எண்களைக் குறிக்கின்றன.) மண்டையறை எலும்பு</s>
<s> மனித உடலில் எத்தனை எலும்புகள் உள்ளன?</s></s>s) பிரிக்கப்பட்டுள்ளன. (தடித்த எண்கள் அருகிலுள்ள படத்தில்

In [190]:

tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 4), (5, 7), (7, 11), (12, 18), (19, 20), (20, 24), (24, 26), (26, 29), (30, 35), (35, 36), (0, 0), (0, 0), (0, 3), (4, 10), (11, 15), (15, 18), (19, 23), (23, 24), (24, 28), (29, 30), (30, 34), (34, 36), (36, 41), (41, 42), (43, 47), (47, 52), (53, 56), (57, 58), (58, 60), (60, 62), (62, 64), (64, 68), (68, 70), (71, 77), (78, 83), (83, 87), (87, 89), (90, 94), (94, 100), (100, 103), (104, 107), (107, 108), (109, 118), (118, 121), (122, 123), (123, 127), (127, 129), (129, 134), (135, 140), (140, 149), (149, 150), (151, 155), (156, 165), (166, 168), (168, 170), (170, 172), (172, 175), (175, 179), (180, 184), (184, 188), (188, 193), (194, 201), (201, 202), (203, 205), (205, 207), (207, 209), (209, 214), (214, 217), (217, 218), (219, 228), (228, 230), (230, 234), (234, 235), (235, 236), (237, 240), (240, 242), (243, 250), (251, 260), (260, 263), (264, 270), (270, 275), (275, 276), (277, 280), (281, 284), (284, 286), (286, 287), (288, 290), (290, 292), (293, 294), (294, 298), 

In [191]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

▁மனித மனித


In [192]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, None]


In [193]:
example

{'id': '903deec17',
 'context': 'ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின்வரும் 206 (மார்பெலும்பு மூன்று பகுதிகளாகக் கருதப்பட்டால் 208) எண்ணிக்கையான எலும்புகளைக் கொண்டிருக்கும். இந்த எண்ணிக்கை உடற்கூட்டியல் வேறுபாடுகளைப் பொறுத்து மாறுபடக்கூடும். எடுத்துக்காட்டாக, மிகக் குறைந்த எண்ணிக்கையான மனிதர்களில், ஒரு மேலதிக விலா எலும்பு (கழுத்துவில்) அல்லது ஒரு மேலதிகமான கீழ் முதுகெலும்பு காணப்படுவதுண்டு; இணைந்த சில எலும்புகளைத் தனி எலும்பாகக் கருதாவிடின், ஐந்து இணைந்த திருவெலும்பு; மூன்று (3 - 5) குயிலலகு எலும்புகள் சேர்ந்து 26 எண்ணிக்கையிலான முதுகெலும்புகள் 33 ஆகக் கருதப்படலாம்.\nமனித மண்டையோட்டில் 22 எலும்புகள் (காதுச் சிற்றென்புகளைத் தவிர) உள்ளன; இவை எட்டு மண்டையறை (cranium) எலும்புகளாகவும் 14 முக எலும்புகளாகவும் (facial bones) பிரிக்கப்பட்டுள்ளன. (தடித்த எண்கள் அருகிலுள்ள படத்தில் காணும் எண்களைக் குறிக்கின்றன.)\n\n\n\nமண்டையறை எலும்புகள் (8)\n 1 நுதலெலும்பு (frontal bone) \n 2 சுவரெலும்பு (parietal bone) (2)\n 3 கடைநுதலெலும்பு (temporal bone) (2)\n 4 பிடர் எலும்பு (occipital bone)\n ஆப

In [194]:
#answers = example["answers"]
start_char = example["answer_start"] 
end_char = start_char + len(example["answer_text"])


# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

27 27


In [195]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print( example["answer_text"])

206
206


In [196]:
pad_on_right = tokenizer.padding_side == "right"

In [197]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []
    
    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        #answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if examples["answer_start"][sample_index]==0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = examples["answer_start"][sample_index]
            end_char = start_char + len(examples["answer_text"][sample_index])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1
            
            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [198]:
tokenized_example = prepare_train_features(datasets['train'][:1])
tokenized_example

{'input_ids': [[0, 69535, 81049, 37368, 153264, 12095, 52989, 21883, 1629, 145615, 32, 2, 2, 3219, 224013, 124335, 5966, 69535, 4930, 74149, 12095, 52989, 21883, 182394, 3686, 51833, 57210, 101912, 15, 6161, 2912, 70597, 52989, 21883, 102080, 54512, 91585, 1962, 212933, 18599, 16242, 94236, 16, 198236, 29160, 12095, 52989, 21883, 173139, 23618, 72817, 5, 5894, 198236, 81049, 37334, 144257, 7827, 82890, 84853, 80517, 114452, 232094, 3686, 17984, 11830, 62001, 182394, 4167, 5, 203312, 10753, 50667, 2650, 4, 45303, 1962, 163062, 198236, 29160, 176030, 15453, 4, 3219, 171093, 5944, 2650, 8120, 10175, 12095, 52989, 21883, 15, 2650, 24183, 5638, 14861, 16, 56735, 3219, 171093, 5944, 2650, 12009, 145578, 10832, 2802, 2650, 26873, 52989, 21883, 53336, 26415, 38640, 31067, 74, 105457, 5966, 22050, 12095, 52989, 21883, 202342, 59386, 12095, 52989, 13070, 2650, 1962, 39311, 9654, 37964, 18806, 4, 196586, 105457, 5966, 22467, 78876, 52989, 21883, 74, 102080, 6896, 20, 21162, 14233, 94097, 4864, 12

In [199]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

In [200]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

loading configuration file ./xlm-roberta/config.json
Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading weights file ./xlm-roberta/pytorch_model.bin
Some weights of the model checkpoint at ./xlm-roberta were not used when initializing XLMRobertaForQuestionAnswering: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.wei

In [201]:
import os
os.environ["WANDB_DISABLED"] = "true"
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"xlm-roberta-finetune",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    save_steps=10000,
    push_to_hub=False,
    #push_to_hub_model_id=f"{model_name}-finetuned-squad",
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [202]:

from transformers import default_data_collator

data_collator = default_data_collator

In [203]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    #model_init='./india-qa-xlm-roberta/',
    #use_auth_token=True
)

In [204]:
trainer.train()

***** Running training *****
  Num examples = 17424
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 21780


Epoch,Training Loss,Validation Loss
1,0.472400,0.292774
2,0.466400,0.156152
3,0.248400,0.103739
4,0.143100,0.060221
5,0.059300,0.037970


***** Running Evaluation *****
  Num examples = 17424
  Batch size = 4
***** Running Evaluation *****
  Num examples = 17424
  Batch size = 4
Saving model checkpoint to xlm-roberta-finetune/checkpoint-10000
Configuration saved in xlm-roberta-finetune/checkpoint-10000/config.json
Model weights saved in xlm-roberta-finetune/checkpoint-10000/pytorch_model.bin
tokenizer config file saved in xlm-roberta-finetune/checkpoint-10000/tokenizer_config.json
Special tokens file saved in xlm-roberta-finetune/checkpoint-10000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 17424
  Batch size = 4
***** Running Evaluation *****
  Num examples = 17424
  Batch size = 4
Saving model checkpoint to xlm-roberta-finetune/checkpoint-20000
Configuration saved in xlm-roberta-finetune/checkpoint-20000/config.json
Model weights saved in xlm-roberta-finetune/checkpoint-20000/pytorch_model.bin
tokenizer config file saved in xlm-roberta-finetune/checkpoint-20000/tokenizer_config.json
Special t

TrainOutput(global_step=21780, training_loss=0.2721441182223233, metrics={'train_runtime': 4727.4308, 'train_samples_per_second': 18.429, 'train_steps_per_second': 4.607, 'total_flos': 1.138208672342016e+16, 'train_loss': 0.2721441182223233, 'epoch': 5.0})

In [205]:
trainer.save_model("india-qa-xlm-roberta")

Saving model checkpoint to india-qa-xlm-roberta
Configuration saved in india-qa-xlm-roberta/config.json
Model weights saved in india-qa-xlm-roberta/pytorch_model.bin
tokenizer config file saved in india-qa-xlm-roberta/tokenizer_config.json
Special tokens file saved in india-qa-xlm-roberta/special_tokens_map.json


In [209]:
import torch

i=0
for batch in trainer.get_eval_dataloader():
    i+=1
    if i>20:
        break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [210]:
output.loss

tensor(1.0937e-05, device='cuda:0')

In [212]:
n_best_size = 20

In [213]:
max_answer_length = 10

In [1]:
import collections
test_datasets = load_dataset('csv', data_files={
                                          'test':'./data/test.csv'})
test_features = test_datasets["test"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=test_datasets["test"].column_names
)
raw_predictions = trainer.predict(test_features)
test_features.set_format(type=test_features.format["type"], columns=list(test_features.features.keys()))

examples = test_datasets["test"]
features = test_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

NameError: name 'load_dataset' is not defined

In [215]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions
            

In [232]:

final_predictions = postprocess_qa_predictions(test_datasets["test"], test_features, raw_predictions.predictions,n_best_size = 30, max_answer_length = 20)

Post-processing 5 example predictions split into 79 features.


In [233]:
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answer_text"]} for ex in datasets["validation"]]
#metric.compute(predictions=formatted_predictions, references=references)

In [234]:
formatted_predictions

[{'id': '22bff3dec', 'prediction_text': 'एम. क्रांति तेलुगु और मां येलन'},
 {'id': '282758170', 'prediction_text': '9 फ़रवरी 2007'},
 {'id': 'd60987e0e', 'prediction_text': '१७ अक्टूबर १८८७'},
 {'id': 'f99c770dc', 'prediction_text': '13'},
 {'id': '40dec1964', 'prediction_text': '1844'}]

In [235]:
wf=open('./data/answer.csv','w',encoding='utf-8')
wf.write(",".join(['id','PredictionString'])+"\n")
for i  in range(len(formatted_predictions)):
    if formatted_predictions[i]['id']==test_datasets['test']['id'][i]:
        print (test_datasets['test']['id'][i], test_datasets['test']['question'][i],formatted_predictions[i]['prediction_text'] )
        wf.write(",".join([str(test_datasets['test']['id'][i]),str(formatted_predictions[i]['prediction_text'].strip())])+'\n')
wf.close()

22bff3dec ज्वाला गुट्टा की माँ का नाम क्या है एम. क्रांति तेलुगु और मां येलन
282758170 गूगल मैप्स कब लॉन्च किया गया था? 9 फ़रवरी 2007
d60987e0e गुस्ताव किरचॉफ का जन्म कब हुआ था? १७ अक्टूबर १८८७
f99c770dc அலுமினியத்தின் அணு எண் என்ன? 13
40dec1964 இந்தியாவில் பசுமை புரட்சியின் தந்தை என்று கருதப்படுபவர் யார்? 1844


In [236]:
test_datasets['test'][-1]

{'id': '40dec1964',
 'context': 'கூட்டுறவு இயக்க வரலாறு, இங்கிலாந்து  நாட்டில் முதன் முதலாக கூட்டுறவு இயக்கமானது, ரோச்டேல் பயனீர் என்பவரால் சிந்திக்கப்பட்டு, 1844ஆம் ஆண்டில் அவரது நண்பர்கள் மற்றும் நெருங்கியவர்கள் கொண்ட கூட்டம் கூட்டப்பட்டு, அந்த  கூட்டத்தில் தாமே, தங்களுக்குள், தங்கள் தேவைக்காக தங்கள் மூலம் என்ற கொள்கைகளுடன் ரோச்டேல் சமத்துவ முன்னோடிகள் கூட்டுறவு பண்டகசாலைகளை துவக்கினர்.  அக்கூட்டுறவு சங்கத்திற்கான நடைமுறை விதிகள் ஏற்படுத்திக் கொண்டனர். இக்கூட்டுறவு சங்கமே,  பிற நாடுகளில் கூட்டுறவு சங்கங்கள் துவக்க முன்னோடியாக விளங்கியது.[1]\nஜெர்மனியில் 1852ஆம் ஆண்டில் பிரான்ச் ஹெர்மன் சூல்ஸ் (Franz Hermann Schulze) என்பவரது முயற்சியால், நகர் புறங்களில் சிக்கன நாணயக் கூட்டுறவு கடன் சங்கங்கள் துவக்கப்பட்டது. 1864ஆம் ஆண்டில் ரெய்பிசன் (Raiffeisen) என்பவரது முயற்சியால் கிராமப்புறங்களில் கூட்டுறவு கடன் சங்கங்கள் துவக்கப்பட்டது. ரெய்பிசனின் வெற்றியைத் தொடர்ந்து,  இங்கிலாந்தில் கூட்டுறவு வீட்டு வசதி சங்கங்கள் மற்றும் பரஸ்பர சேமிப்பு கூட்டுறவு வங்கிகள் துவக்கப்பட்டது. பிரான்சில் அச்சுத் தொழ